In [2]:
## create .node file for brain map visulization:

import os
# import shutil
from glob import glob
import numpy as np
import pandas as pd
from natsort import natsorted
import seaborn as sns
from tqdm import tqdm, trange
import matplotlib.pyplot as plt


In [27]:

in_dir1       = "data_subject_info/"
in_file_name1 = "target_brain_env_mni.csv"

in_dir2       = "out01_tms_induced_respose/"
in_file_name2 = "HRF_tms_induced_response.csv"
out_dir       = "out02_brain_map/"

if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    
df_env_mni      = pd.read_csv(in_dir1 + in_file_name1)
df_tms_response = pd.read_csv(in_dir2 + in_file_name2)

# replace site label as they are not consistent across two files to merge:
tms_sites = ["L_Fp","R_Fp","L_aMFG","R_aMFG","L_pMFG","R_pMFG","R_IFJ","R_FEF","R_M1","R_preSMA","R_IPL"]

# print the order of site to make sure they are consistent with the above labels:
curr_label = list(df_env_mni['site'].drop_duplicates())
print(curr_label)

df_env_mni['site'].replace(curr_label, tms_sites, inplace = True)
df_tms_response = df_tms_response.merge(df_env_mni, on = ['subject', 'site'], how = 'inner')
df_tms_response.drop(labels = ['inputfile', 'subject', 'group'], axis = 1, inplace = True)

['L-FP', 'R-FP', 'L-aMFG', 'R-aMFG', 'L-pMFG', 'R-pMFG', 'R-IFJ', 'R-FEF', 'R-M1', 'R-preSMA', 'R-IPL']


In [28]:
df_tms_response['site_index']  = df_tms_response['site'].replace(tms_sites, list(range(1, 12)))
df_tms_response

,site,tms_site_response_standard,tms_site_response_individual_6mm,tms_site_response_individual_10mm,tms_site_response_individual_14mm,tms_site_response_individual_14-10mm,tms_site_response_individual_10-6mm,env_mni_x,env_mni_y,env_mni_z,site_index
0,L_Fp,0.287125,0.439684,0.456288,0.395807,0.354767,0.462649,-28,64,6,1
1,L_Fp,-1.260076,1.178456,0.916780,0.524276,0.390585,0.894974,-28,68,24,1
2,L_Fp,-1.618781,-1.208849,-1.281960,-1.378883,-1.432572,-1.302709,-20,70,-10,1
3,L_Fp,-0.523261,0.030765,0.065123,-0.066875,-0.118009,0.069139,-22,70,-16,1
4,L_Fp,0.764099,0.906085,0.901252,0.830509,0.790728,0.899914,-34,66,0,1
...,...,...,...,...,...,...,...,...,...,...,...
690,R_preSMA,0.214189,-0.228209,-0.128205,0.078518,0.208812,-0.092598,4,-2,72,10
691,R_preSMA,0.155220,0.220501,0.183733,0.107853,0.057474,0.168976,6,2,70,10
692,R_preSMA,-0.534299,-1.183636,-0.868778,-0.557893,-0.361947,-0.756670,4,-2,72,10
693,R_preSMA,-0.217289,-1.108896,-0.575760,-0.114780,0.170221,-0.383482,4,2,72,10


In [74]:
test_variables = ['tms_site_response_standard', 'tms_site_response_individual_6mm',
                  'tms_site_response_individual_10mm', 'tms_site_response_individual_14mm', 
                  'tms_site_response_individual_14-10mm', 'tms_site_response_individual_10-6mm']

for var in test_variables:
    node_file = df_tms_response[['env_mni_x', 'env_mni_y', 'env_mni_z']]
    node_file['node_color'] = [1 if val > 0 else 0 for val in df_tms_response[var]]
    node_file['node_size'] = 1
    
    node_file.to_csv(out_dir + "node_mni_binarized_" + var + ".node", 
                     sep = '\t', header = False, index = False)

<ipython-input-74-b1b70ed38df4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  node_file['node_color'] = [1 if val > 0 else 0 for val in df_tms_response[var]]
<ipython-input-74-b1b70ed38df4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  node_file['node_size'] = 1


In [75]:
node_file

,env_mni_x,env_mni_y,env_mni_z,node_color,node_size
0,-28,64,6,1,1
1,-28,68,24,1,1
2,-20,70,-10,0,1
3,-22,70,-16,1,1
4,-34,66,0,1,1
...,...,...,...,...,...
690,4,-2,72,0,1
691,6,2,70,1,1
692,4,-2,72,0,1
693,4,2,72,0,1
